In [64]:
import pandas as pd
import glob
import re
fs = glob.glob("/home/giogio/oxford_3000/OLS/my_import_*.xlsx")

all_data = []
all_df = []

d_pattern = re.compile(r'(\d\))')
for file in fs:
    # Excelファイルを読み込む
    df = pd.read_excel(file)
    all_df.append(df)


    for index, row in df.iterrows():
        term = row["term"].strip()
        text = row["definition"].strip()
        
        parts = d_pattern.split(text)
        for part in parts:
            part2 = part.split("\n")
            definition = part2[0]
            if len(definition)<=3:
                continue
            usage = "\n".join(part2[1:])
            all_data.append({
                'term': term,
                'definition': definition.strip(),
                'usage': usage.strip("\n").replace("_ "*len(term),term)
            })
        
# データフレームを作成
result_df = pd.DataFrame(all_data, columns=['term', 'definition', 'usage'])

# 結果を表示
print(result_df)

# # 結果を新しいExcelファイルに保存
result_df.dropna().to_excel('processed_terms.xlsx', index=False)


           term                                         definition  \
0       respond  to give a spoken or written answer to someone ...   
1       respond  to do something as a reaction to something tha...   
2       respond  to react quickly or in the correct way to some...   
3      response                         a spoken or written answer   
4      response  a reaction to something that has happened or b...   
...         ...                                                ...   
8888      allow  to let someone or something go into, through, ...   
8889  allow for  to consider or include someone or something wh...   
8890  all right                acceptable; in an acceptable manner   
8891  all right                                      safe and well   
8892  all right                              only just good enough   

                                                  usage  
0     "I asked him his name, but he didn't  respond....  
1     "How did they  respond to the news?"\

In [63]:
result_df["definition"].map(len)

0         0
1         2
2        58
3         2
4        72
         ..
20145    35
20146     2
20147    13
20148     2
20149    21
Name: definition, Length: 20150, dtype: int64

In [54]:
df_all = pd.concat(all_df)
df_all

,term,definition,link,synonyms,idioms,phrasal verbs
0,respond,1) to give a spoken or written answer to someo...,https://www.oxfordlearnersdictionaries.com/sea...,reply\nreact\n,NaN,NaN
1,response,"1) a spoken or written answer\n\n""She made no ...",https://www.oxfordlearnersdictionaries.com/sea...,NaN,NaN,NaN
2,responsibility,1) a duty to deal with or take care of someone...,https://www.oxfordlearnersdictionaries.com/sea...,NaN,NaN,NaN
0,flooded,"1) if a place floods or something floods it, i...",https://www.oxfordlearnersdictionaries.com/sea...,pour\n,NaN,NaN
1,flooding,"1) if a place floods or something floods it, i...",https://www.oxfordlearnersdictionaries.com/sea...,pour\n,NaN,NaN
...,...,...,...,...,...,...
1,ocean,1) the mass of salt water that covers most of ...,https://www.oxfordlearnersdictionaries.com/sea...,NaN,an ocean of something - (informal) a large amo...,NaN
2,o’clock,used with the numbers 1 to 12 when telling the...,https://www.oxfordlearnersdictionaries.com/sea...,NaN,NaN,NaN
0,allow,1) to let someone or something do something; t...,https://www.oxfordlearnersdictionaries.com/sea...,NaN,allow me - used to offer help politely\n“Allo...,NaN
1,allow for,to consider or include someone or something wh...,https://www.oxfordlearnersdictionaries.com/sea...,NaN,NaN,NaN


In [66]:
print(df_all.query("term=='sell off'")["definition"].values[0])

1) to sell things cheaply because you want to get rid of them or because you need the money



2) to sell all or part of an industry, a company, or land

"The Church sold off the land for housing."
"In the eighties many state-owned businesses in the U.K. were sold off.">


In [ ]:
import pandas as pd
import glob
fs = glob.glob("/home/giogio/oxford_3000/OLS/my_import_*.xlsx")

all_data = []
all_df = []
for file in fs:
    # Excelファイルを読み込む
    df = pd.read_excel(file)
    all_df.append(df)
    
    
    # 定義と使用例を分割するためのフラグ
    is_definition = True
    definition = ''
    usage_list = []

    for index, row in df.iterrows():
        term = row["term"].strip()
        text = row["definition"].strip()
        for line in text.split("\n"):
            line = line.strip()
            if line.startswith('1)') or line.startswith('2)') or line.startswith('3)'):
                # 以前の定義と使用例を保存
                if definition and usage_list:
                    usage = "\n".join(usage_list)
                    all_data.append({
                        'term': term,
                        'definition': definition,
                        'usage': usage
                    })
                    # リセット
                    usage_list = []

                # 新しい定義部分
                definition = line
            else:
                # 使用例部分
                line2 = line.replace("_ "*len(term),term)
                usage_list.append(line2)
    
    # 最後の定義と使用例を保存
    if definition and usage_list:
        usage = "\n".join(usage_list)
        all_data.append({
            'term': term,
            'definition': definition,
            'usage': usage
        })
        usage_list = []

# データフレームを作成
result_df = pd.DataFrame(all_data, columns=['term', 'definition', 'usage'])

# 結果を表示
print(result_df)

# # 結果を新しいExcelファイルに保存
result_df.to_excel('processed_terms.xlsx', index=False)
